In [4]:
import cv2
import numpy as np
import mediapipe as mp
import time
import math
import pyttsx3


# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
mp_drawing = mp.solutions.drawing_utils

# Eye landmarks from MediaPipe (based on iris tracking)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [263, 387, 385, 362, 380, 373]

engine = pyttsx3.init()

# EAR calculation function
def calculate_ear(landmarks, eye_indices, img_w, img_h):
    p = [landmarks[i] for i in eye_indices]
    coords = [(int(p[i].x * img_w), int(p[i].y * img_h)) for i in range(6)]
    # Vertical distances
    v1 = np.linalg.norm(np.array(coords[1]) - np.array(coords[5]))
    v2 = np.linalg.norm(np.array(coords[2]) - np.array(coords[4]))
    # Horizontal distance
    h = np.linalg.norm(np.array(coords[0]) - np.array(coords[3]))
    ear = (v1 + v2) / (2.0 * h)
    return ear

# Thresholds
EAR_THRESHOLD = 0.25
CONSEC_FRAMES = 30  # ~1 second at 30 FPS

frame_counter = 0
alarm_triggered = False

# Start Webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_ear = calculate_ear(face_landmarks.landmark, LEFT_EYE, w, h)
            right_ear = calculate_ear(face_landmarks.landmark, RIGHT_EYE, w, h)
            avg_ear = (left_ear + right_ear) / 2.0

            if avg_ear < EAR_THRESHOLD:
                frame_counter += 1
                if frame_counter > CONSEC_FRAMES and not alarm_triggered:
                    alarm_triggered = True
                    print("⚠️ Drowsiness detected!")
                    cv2.putText(frame, "DROWSINESS ALERT!", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                    engine.say("Wake up! You are feeling drowsy.")
                    engine.runAndWait()
            else:
                frame_counter = 0
                alarm_triggered = False

            # Draw landmarks for visualization (optional)
            mp_drawing.draw_landmarks(
                frame,
                face_landmarks,
                mp_face_mesh.FACEMESH_CONTOURS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

    cv2.imshow("Driver Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


⚠️ Drowsiness detected!
⚠️ Drowsiness detected!
